# 前回の復習・振り返り

相関係数のヒートマップ表示
```
plt.figure(figsize=(10,10))
sns.heatmap(df_one_hot.iloc[:,:20].corr(), cmap="coolwarm_r", vmin=-1, vmax=1, center=0)
```

数値が少ないクラスは、othersでまとめてしまう</br>
その上で2クラスのデータをone hotにしたら、各変数は必ず-1の相関がある</br>
可視化の方法は、y軸:目的変数、x軸:どれかの説明変数</br>
orderedな変数は、カテゴリカル変数にした方が良い、それ以外はone hot変数にした方が良い</br>
SVMとカーネルサイズを試す</br>

# Day2